<a href="https://colab.research.google.com/github/ktynski/Marketing_Automations_Notebooks_With_GPT/blob/main/Intents%2C_Personas%2C_and_Buyer_Inference_(Public).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai pydot graphviz

In [ ]:
import openai
import pandas as pd
import re
import time
import concurrent.futures

# set up OpenAI API credentials
openai.api_key = "Your API Key"


# Define the keyword
keyword = "Home Alarm Systems"
num_personas = 3

# Use GPT3 to predict the intent of the keyword search
prompt = f"What is the most likely intent of searching for {keyword}?"
response = openai.Completion.create(
  engine="text-davinci-003",
  prompt=prompt,
  max_tokens=60,
  n=1,
  stop=None,
  temperature=0.5
)
intent = response.choices[0].text.strip()

# Generate 3 personas for the keyword/intent
prompt = (f"You are an expert at marketing and consumer behavior, specifically persona generation. Given the searcher is using the keyword: \n {keyword} with the intent: \n {intent}, please generate 3 different but unique personas with the following format. Each persona should be seperated by ######## : \n\n"
          "1. Demographics\n"
          "- Name:\n"
          "- Age Range:\n"
          "- Gender:\n"
          "- Marital Status:\n"
          "- Income:\n"
          "- Education Level:\n\n"
          "2. Psychographics\n"
          "- Personality Traits:\n"
          "- Values and Beliefs:\n"
          "- Interests and Hobbies:\n"
          "- Lifestyle Factors:\n\n"
          "4. Behavior and Decision-Making\n"
          "- Information Sources:\n"
          "- Influences on Purchase Decisions:\n"
          "- Key Behaviors and Habits:\n\n"
          )
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=2000,
    n=1,
    stop=None,
    temperature=0.8
)
personas_text = response.choices[0].text.strip()

# Parse the personas
personas = personas_text.split('########')
personas = [persona.strip() for persona in personas if persona.strip()]

# For each persona, generate the responses to the questions
results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    tasks = []
    for persona in personas:
        for question in [
            f"For the following question please answer with a list. What are the most important features or benefits that a given persona is looking for when searching for {keyword}, given a persona of: \n Persona:\n {persona} ?",
            f"For the following question please answer with a list. What are the most common questions or concerns before making a purchase related to {keyword}, given a persona of: \n Persona:\n {persona} ?",
            f"For the following question please answer with a list. What are the biggest challenges that someone faces when searching for the right product or service related to {keyword}, given a persona of: \n Persona: \n {persona} ?",
            f"For the following question please answer with a list. How does the given persona typically research their options related to {keyword}, and where do they go to find information, given a persona of: \n Persona: \n {persona}?",
            f"For the following question please answer with a list. What are the most effective ways to convince the following persona to make a purchase related to {keyword}, given a persona of: \n Persona: \n {persona} ?",
            f"For the following question please answer with a list. What are the most common objections when considering a purchase related to {keyword}, and how can they be addressed given a persona of: \n Persona: \n {persona}",
            f"For the following question please answer with a list. What are some unique or unconventional ways to market products or services related to {keyword} given a persona of: \n Persona: \n {persona} ?",
            f"For the following question please answer with a list. How does the given persona typically compare different products or services related to {keyword}, and what factors do they consider? \n Persona: \n {persona}",
            f"For the following question please answer with a list. What are the most common misconceptions or misunderstandings related to {keyword}, and how can they be corrected? given the persona of: \n Persona: \n {persona}",
            f"For the following question please answer with a list. What are some emerging trends or developments in the market for {keyword}, and how can they leverage them to gain an advantage given the persona of: \n Persona: \n {persona}?"
        ]:
            prompt = question
            task = executor.submit(openai.Completion.create,
              engine="text-davinci-003",
              prompt=prompt,
              max_tokens=200,
              n=1,
              stop=None,
              temperature=0.5
            )
            tasks.append((persona, question, task))
    for persona, question, task in tasks:
        response = task.result()
        answer = response.choices[0].text.strip()
        results.append({
            "Persona": persona,
            "Question": question,
            "Answer": answer
        })

# Collate everything into a neatly organized dataframe
df = pd.DataFrame(results)
print(df)


In [ ]:
df.to_csv("buyer_journey11.csv")